# 双向最大匹配

https://blog.csdn.net/Elenore1997/article/details/83274720

https://blog.csdn.net/weixin_43256799/article/details/86098695

算法描述
正向最大匹配算法
先设定扫描的窗口大小maxLen（最好是字典最长的单词长度），从左向右取待切分汉语句的maxLen个字符作为匹配字段。查找词典并进行匹配。若匹配成功，则将这个匹配字段作为一个词切分出来，并将窗口向右移动这个单词的长度。若匹配不成功，则将这个匹配字段的最后一个字去掉，剩下的字符串作为新的匹配字段，进行再次匹配，重复以上过程，直到切分出所有词为止。

反向最大匹配算法
该算法是正向的逆向算法，区别是窗口是从后向左扫描，若匹配不成功，则去掉第一个字符，重复上述的匹配步骤。

双向最大匹配算法
双向最大匹配法是将正向最大匹配法得到的分词结果和逆向最大匹配法的到的结果进行比较，从而决定正确的分词方法。定义的匹配规则如下：

如果正反向匹配算法得到的结果相同，我们则认为分词正确，返回任意一个结果即可。
如果正反向匹配算法得到的结果不同，则考虑单字词、非字典词、总词数数量的数量，三者的数量越少，认为分词的效果越好。我们设定一个惩罚分数（score_fmm / score_bmm = 0），例如：正向匹配中单字词数量多于反向匹配，则正向匹配的分值score_fmm += 1。其他两个条件相同。可以根据实际的分词效果调整惩罚分数的大小，但由于没有正确分词的数据，因此惩罚分数都设为1。最后比较惩罚分数，返回较小的匹配结果。


In [49]:
def fmm(text, dic):
    res = []
    index = 0
    window_size = max([len(w) for w in dic])
    while index < len(text):
        for size in range(window_size, 0, -1):
            piece = text[index:index + size]
            if piece in dic:
                index = index + size - 1
                break
        index += 1
        res.append(piece)
    return res

In [50]:
def bmm(text, dic):
    res = []
    index = len(text)
    window_size = max([len(w) for w in dic])
    while index > 0:
        for size in range(window_size, 0, -1):
            piece = text[index - size:index]
            if piece in dic:
                index = index - size + 1
                break
        index -= 1
        res.append(piece)
    return res[::-1]

In [51]:
def bimm(text, dic):
    fw = fmm(text, dic)
    bw = bmm(text, dic)
    # 正反向分词结果
    print("FMM: ", fw)
    print("BMM: ", bw)
    # 单字词个数
    f_single_word = 0
    b_single_word = 0
    # 总词数
    tot_fmm = len(fw)
    tot_bmm = len(bw)
    # 非字典词数
    oov_fmm = 0
    oov_bmm = 0
    # 罚分，罚分值越低越好
    score_fmm = 0
    score_bmm = 0
    # 如果正向和反向结果一样，返回任意一个
    if fw == bw:
        return fw
    else:  # 分词结果不同，返回单字数、非字典词、总词数少的那一个
        for w in fw:
            if len(w) == 1:
                f_single_word += 1
            if w not in dic:
                oov_fmm += 1
        for w in bw:
            if len(w) == 1:
                b_single_word += 1
            if w not in dic:
                oov_bmm += 1
        # 可以根据实际情况调整惩罚分值
        # 这里都罚分都为1分
        # 非字典词越少越好
        if oov_fmm > oov_bmm:
            score_fmm += 1
        elif oov_fmm < oov_bmm:
            score_bmm += 1
        # 总词数越少越好
        if tot_fmm > tot_bmm:
            score_fmm += 1
        elif tot_fmm < tot_bmm:
            score_bmm += 1
        # 单字词越少越好
        if f_single_word > b_single_word:
            score_fmm += 1
        elif f_single_word < b_single_word:
            score_bmm += 1

        # 返回罚分少的那个
        if score_fmm < score_bmm:
            return fw
        else:
            return bw

In [52]:
dic = ['研究','研究生','生命','命','的','起源']
text = '研究生命的起源'

In [53]:
bimm(text, dic)

FMM:  ['研究生', '命', '的', '起源']
BMM:  ['研究', '生命', '的', '起源']


['研究', '生命', '的', '起源']